<a href="https://colab.research.google.com/github/harshrojivadia/coding-assigment/blob/master/ResnetExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://www.kaggle.com/hafizurrahman/flower-classification-resnet50-transfer-learning

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"harshrojivadia","key":"0417e27a83c184b2c8e26fa8ad0f25ab"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d alxmamaev/flowers-recognition

 98% 441M/450M [00:03<00:00, 227MB/s]
100% 450M/450M [00:03<00:00, 151MB/s]


In [ ]:
!unzip \*.zip

In [ ]:
import os
import numpy as np
import cv2
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

In [ ]:
arr = os.listdir('./flowers/')
print(arr)
print(len(arr))

['tulip', 'daisy', 'flowers', 'rose', 'dandelion', 'sunflower']
6


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import shutil
import matplotlib.pyplot as plt
%matplotlib inline 

from IPython.display import Image, display
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16 , preprocess_input
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

import os

In [ ]:
base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(224, 224, 3))

# base_model = VGG16(weights='imagenet', 
#                       include_top=False, 
#                       input_shape=(224, 224, 3))

In [ ]:
# Create model
model = Sequential()
model.add(ResNet50(include_top=False, pooling='avg', weights="imagenet"))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(5, activation='softmax'))

# Do not train first layer (ResNet) as it is already pre-trained
model.layers[0].trainable = False

# Compile model
from tensorflow.keras import optimizers

# sgd = optimizers.SGD(lr = 1e-4, decay = 1e-6, momentum = 0.9, nesterov = True)
# adam = optimizers.Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.004)
# sgd = optimizers.SGD(lr = 0.01, nesterov = True)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
print(model.layers[0])
print(model.layers)

[<tensorflow.python.keras.engine.functional.Functional object at 0x7fdece13fa50>, <tensorflow.python.keras.layers.core.Flatten object at 0x7fdec7c55290>, <tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x7fdece0558d0>, <tensorflow.python.keras.layers.core.Dense object at 0x7fdec7f9fbd0>, <tensorflow.python.keras.layers.core.Dropout object at 0x7fdece2b24d0>, <tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x7fdece249590>, <tensorflow.python.keras.layers.core.Dense object at 0x7fdec6541550>, <tensorflow.python.keras.layers.core.Dropout object at 0x7fdec67248d0>, <tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x7fdebcffbdd0>, <tensorflow.python.keras.layers.core.Dense object at 0x7fdec7ae5610>]


In [ ]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
flatten_14 (Flatten)         (None, 2048)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 2048)              8192      
_________________________________________________________________
dense_24 (Dense)             (None, 2048)              4196352   
_________________________________________________________________
dropout_12 (Dropout)         (None, 2048)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 2048)              8192      
_________________________________________________________________
dense_25 (Dense)             (None, 1024)            

In [ ]:
image_size = 224
# d5ont do rescale it is affecting validation acc badly
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input,
                                    horizontal_flip=True,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    validation_split=0.2)# set validation split


In [ ]:
train_generator = data_generator.flow_from_directory(
    './flowers/flowers',
    target_size=(image_size, image_size),
    batch_size=24,
    class_mode='categorical',
    subset='training'
    )
validation_generator = data_generator.flow_from_directory(
    './flowers/flowers',
    target_size=(image_size, image_size),
    batch_size=24,
    class_mode='categorical',
    subset='validation'
    )

Found 3462 images belonging to 5 classes.
Found 861 images belonging to 5 classes.


In [ ]:
NUM_EPOCHS = 100
import math

fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=10,
    validation_data=validation_generator,
    validation_steps=10,
    epochs=NUM_EPOCHS)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
10/10 [==============================] - 6s 656ms/step - loss: 0.6024 - accuracy: 0.8083 - val_loss: 0.4289 - val_accuracy: 0.8750
Epoch 2/100
10/10 [==============================] - 6s 662ms/step - loss: 0.6766 - accuracy: 0.8042 - val_loss: 0.5810 - val_accuracy: 0.8167
Epoch 3/100
10/10 [==============================] - 6s 654ms/step - loss: 0.5733 - accuracy: 0.8125 - val_loss: 0.6234 - val_accuracy: 0.8333
Epoch 4/100
10/10 [==============================] - 6s 660ms/step - loss: 0.5890 - accuracy: 0.8125 - val_loss: 0.6287 - val_accuracy: 0.8500
Epoch 5/100
10/10 [==============================] - 6s 655ms/step - loss: 0.6166 - accuracy: 0.8000 - val_loss: 0.5265 - val_accuracy: 0.8625
Epoch 6/100
10/10 [==============================] - 6s 658ms/step - loss: 0.7054 - accuracy: 0.8292 - val_loss: 0.4415 - val_accuracy: 0.8583
Epoch 7/100
10/10 [==============================] - 6s 667ms/step - loss: 0.5036 - accuracy: 0.8292 - val_loss: 0.5064 - val_accuracy: 0.8417

KeyboardInterrupt: ignored